# Ant-Colony-Optimization Problem

This Jupyter Notebook deals with Ant-Colony-Optimization. All information is taken from Daniel Merkle's and Martin Middendorf's Chapter 14 on Swarm Intelligence in Search Methodologies. Introductory Tutorials in Optimization and Decision Support Techniques by Edmund K. Burke and Graham Kendall.

Ant-Colony Optimization is one of two main areas of Swarm Intelligence. It is used to solve optimization and search problems. The idea is that ant colonies find the shortest way form their nest to the food source with the help of pheromones. Experiments have shown that if you connect the nest with two different branches to a food source - one way is longer than the other one, ants will quickly find out which way is the shortest one. The result is an ant trail.

How exactly does it work? At the beginning both branches are taken with the same probability. After a while the pheromone concentration will be higher at the shorter branch since the ants are back at the nest way faster if they have taken the short road. The pheromone concentration gets higher more easily and since ants are taking the road with a higher pheromone concentration, eventually all ants take the short road

In [16]:
import pandas as pd
import numpy as np

## 1. Initialization
Importing .txt files and saving them as arrays.

In [17]:
with open('01_tsp.txt', 'r') as t1:
    array1 = []
    for line in t1.readlines():
        y = [int(value) for value in line.split()]
        array1.append(y)

    t1.close()
    array1 = np.array(array1)

with open('02_tsp.txt', 'r') as t2:
    array2 = []
    for line in t2.readlines():
        y = [int(value) for value in line.split()]
        array2.append(y)

    t2.close()
    array2 = np.array(array2)

with open('03_tsp.txt', 'r') as t3:
    array3 = []
    for line in t3.readlines():
        y = [int(value) for value in line.split()]
        array3.append(y)

    t3.close()
    array3 = np.array(array3)

# Test: Print three imported arrays, their shape and check if it is a numpy array
# print(array1)
# print(array2)
# print(array3)
# print(array1.shape)
# print(type(array1))

pheromone_matrix = np.zeros_like(array1)

## 2. Solution Construction

In [18]:
def solution_construction(distance_matrix, pheromone_matrix, number_ants):
    citiesleft = True
    #for each ant
    while number_ants
        #start with a random city
        
        
        while citiesleft:
        #select next city until no item left
        
            #with probability p and heuristics
            

            #remove city after selection
            #stop if no cities left
            if selection == False:
                citiesleft = False
            else:
                pass
        #
        number_ants -= 1

SyntaxError: invalid syntax (<ipython-input-18-a2e7ab0f8f2d>, line 4)

## 3. Evaporation

In [19]:
def evaporate(pheromone_matrix):
    pheromone_matrix = pheromone_matrix * 0.1

    return pheromone_matrix

evaporate(array1)

array([[ 0. , 10. , 22.9, ..., 37.9, 18.3, 17.8],
       [10. ,  0. , 32.9, ..., 47.9, 23.1, 27.8],
       [22.9, 32.9,  0. , ..., 15. , 29. ,  8.7],
       ...,
       [37.9, 47.9, 15. , ...,  0. , 24.8, 20.1],
       [18.3, 23.1, 29. , ..., 24.8,  0. , 20.3],
       [17.8, 27.8,  8.7, ..., 20.1, 20.3,  0. ]])

## 4. Intensification

solution_matrix will be a list of lists which each holding the numbers of the cities in the order that  each of the ants visited them


In [11]:
# array2=[[1, 1, 1],[2, 2, 2],[3, 3, 1]]
# array3=[[1, 2, 0],[2, 2, 2]]
# array0=[[0, 4, 0],[0, 0, 4],[4, 0, 0]]

def intensification(distance_matrix, pheromone_matrix, solution_matrix):
      
    shortest_distance = 10000000000
    for i in solution_matrix:
    
        distance=0
        y=0
       
        for j in i:
            distance= distance + distance_matrix[y][j]
            y=j
        
        if distance < shortest_distance:
            shortest_distance=distance
            #print(shortest_distance)
            best_solution=i
            #print(best_solution)


    x=0
    for k in best_solution:
          
        pheromone_matrix[x][k]= pheromone_matrix[x][k]*1.5
        x=k
    return pheromone_matrix
            
#print(
#intensification(array0,array2,array3)

12
[1, 2, 0]
0
[2, 2, 2]


[[1, 1, 1.5], [2, 2, 2], [3, 3, 2.25]]

## Action